In [48]:
def get_nora3(latlons, vars, start, end, testing= True):

    import pandas as pd
    import netCDF4
    import pyproj
    import numpy as np
    import itertools

    start_date = start
    end_date = end

    timerange = pd.date_range(start=start_date, end = end_date, freq = 'H')
    timerange_6 = pd.date_range(start=start_date, end = end_date, freq = '6H')
    timeset = ['03','04','05','06','07','08','09']

    for hind,ind in zip(itertools.cycle(timeset),range(len(timerange_6))):
        date = timerange_6.date[ind].strftime("%Y%m%d")
        time = timerange_6[int(ind)].strftime("%H")
        filepath_Nora3 = (
                "https://thredds.met.no/thredds/dodsC/nora3/{0}".format(timerange_6.date[ind].strftime("%Y/%m/%d"))
            )   
        try:
            remfile = filepath_Nora3 +"/"+ time +"/fc"+ date + time +"_0" + hind + ".nc"
            
            DS = netCDF4.Dataset(remfile,"r")
            print(DS.variables["time"])
            x = DS.variables["x"][:]
            y = DS.variables["y"][:]

            proj = pyproj.Proj(DS.variables["projection_lambert"].proj4) # get projection to lat long as function

            #dates = pd.Series(np.full(horizon, fill_value=DS.min_time))
            #time_ahead = dates.index

            A  = np.zeros((len(vars),len(latlons))) * np.nan

            for varidx, var in enumerate(vars): # need to make a manual var list
                a = DS.variables[var][0,0,:,:]# results from format (1,1,x,y)
                print(DS.variables[var])
                for ptidx, latlon in enumerate(latlons):
                    lat = latlon[0]
                    lon = latlon[1]
                    # Compute projected coordinates of lat/lon point
                    X,Y = proj(lon,lat)
                    Ix = np.argmin(np.abs(x - X)) # take location which is closest
                    Iy = np.argmin(np.abs(y - Y)) 
                    A[varidx,ptidx] = a[Iy,Ix] 
            
            if not testing:
                for ptidx, latlon in enumerate(latlons): # data saving format, apart from file name should be good
                    df = pd.DataFrame(A[:,ptidx])
                    #df.index = pd.MultiIndex.from_arrays([dates, time_ahead], names=['Time', 'horizon'])
                    df.columns = vars
                    filename = "nora3_lat{0}_lon{1}_{2}_{3}.csv".format(latlon[0], latlon[1], date, time)
                    df.to_csv(filename)
            else: 
                for ptidx, latlon in enumerate(latlons): # data saving format, apart from file name should be good
                    df = pd.DataFrame(A[:,ptidx])
                    #df.index = pd.MultiIndex.from_arrays([dates, time_ahead], names=['Time', 'horizon'])
                    print(df)
                    df.columns = vars
                    filename = "nora3_lat{0}_lon{1}_{2}_{3}.csv".format(latlon[0], latlon[1], date, time)
                    print(filename)
       
        except:
            raise "well this isnt supposed to happen though.."
            #continue
    
    
            







In [29]:
import pandas as pd
import netCDF4
import pyproj
import numpy as np
import itertools
start_date = "01-01-2021"
end_date = "01-07-2021"

timerange = pd.date_range(start=start_date, end = end_date, freq = 'H')
timerange_6 = pd.date_range(start=start_date, end = end_date, freq = '6H')
timeset = ['03','04','05','06','07','08','09']

In [42]:
count = 0
for horizon,ind in zip(itertools.cycle(timeset),range(len(timerange_6))):
    date = timerange_6.date[ind].strftime("%Y%m%d")
    time = timerange_6[int(ind)].strftime("%H")
    #horizon = timerange[ind].strftime("%H")
    filepath_Nora3 = (
            "https://thredds.met.no/thredds/dodsC/nora3/{0}".format(timerange.date[ind].strftime("%Y/%m/%d"))
        )   
  
    remfile = filepath_Nora3 +"/"+ time +"/fc"+ date + time +"_0" + horizon + "_fp.nc"
    print(remfile)
    count +=1
print (count)

NameError: name 'itertools' is not defined

In [3]:
import netCDF4
import pyproj
import pandas as pd
import numpy as np

In [11]:
# use this as latlon

latlons = [[58.134, 7.0452],
 [58.34, 8.5225],
 [58.7605, 5.6508],
 [59.0467, 10.125],
 [59.2268, 9.5772],
 [59.3188, 10.8145],
 [59.3193, 11.0422],
 [59.3808, 10.2397],
 [59.3882, 11.3905],
 [59.6193, 10.215],
 [59.6605, 10.7818],
 [59.6625, 5.9538],
 [59.7908, 10.2598],
 [59.9423, 10.72],
 [60.1268, 11.3933],
 [60.1403, 10.2662],
 [60.3185, 6.6538],
 [60.3558, 10.5592],
 [60.3837, 5.332],
 [60.7002, 10.8695]]

In [27]:
vars = [
"integral_of_surface_net_downward_shortwave_flux_wrt_time",
"air_temperature_2m",
"relative_humidity_2m",
"cloud_area_fraction",
"surface_air_pressure",
"wind_direction",
"wind_speed",
"snowfall_amount_acc"]
latlons = [[60.7008, 10.8678], 
           [63.3405, 10.2981]]
#"x_wind_10m",
#"y_wind_10m",
#"x_wind_gust_10m",
#"y_wind_gust_10m",

In [2]:
vars = ["surface_downwelling_shortwave_flux_in_air","air_temperature_2m","wind_speed_of_gust"]
latlons = [[60.7008, 10.8678], 
           [63.3405, 10.2981]]

In [49]:
import time

start = time.time()

get_nora3(latlons, vars, start='2020-10-19', end='2021-12-31')

end = time.time()
diff = end-start
print(diff)

<class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    long_name: time
    standard_name: time
    units: seconds since 1970-01-01 00:00:00 +00:00
    _ChunkSizes: 1
unlimited dimensions: time
current shape = (1,)
filling off


TypeError: exceptions must derive from BaseException

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2960871502.py, line 1)